# **Google Colab Notebook** 

***

## <span style="color:green">Preprocessing with IQA and VR</span>

### Mounting google drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install dependencies 

- Download CMake

In [ ]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.13.4/cmake-3.13.4.tar.gz"
! tar xf cmake-3.13.4.tar.gz
! cd cmake-3.13.4 && ./configure && make && sudo make install

- Requirements for OpenPose to work on Colab (requires 2 runs)

In [ ]:
# Basic
! sudo -H pip3 install fastapi kaleido python-multipart uvicorn
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy==1.21 protobuf==3.20.3 opencv-python
# OpenCV 2.4 -> Added as option
# # sudo apt-get --assume-yes install libopencv-dev
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

#  Openpose の clone
#! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
! git clone  --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git

#  Openpose の モデルデータDL
! cd openpose/models && ./getModels.sh

# Openpose の ビルド
! sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
! cd openpose && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc`

- Requirements for TOP-IQ (PyTorch implementation for IQA)

In [ ]:
!cd ./drive/MyDrive/BTP/IQA-PyTorch && pip3 install -r requirements.txt && python3 setup.py develop

- Requirements for RVRT

In [ ]:
!cd /content/drive/MyDrive/BTP/RVRT/ && pip3 install -r requirements.txt

- Modules for preprocessing

In [ ]:
%cd ./drive/MyDrive/BTP/IQA-PyTorch
import glob
import tqdm
import os
import cv2
import numpy as np
import pyiqa
import torch
import subprocess
import shutil
%cd '../../../../'

DATASET = 'movies'

### Gamma Correction

In [ ]:
%cd '/content'
GAMMA = 0.67  # if GAMMA < 0, no gamma correction will be applied
# Precompute gamma correction table
gamma_table = np.array([((i / 255.0) ** GAMMA) * 255 for i in np.arange(0, 256)]).astype("uint8")

def ProcessGammaVideo(videoName, inputPath, outputPath):
    print(f'Generating Gamma frames for {videoName}.avi')
    cap = cv2.VideoCapture(inputPath)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(f"{outputPath}/{videoName}.avi", fourcc, fps, (width,  height))
    detected = False
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            if detected:
                print("Video Ended")
            else:
                print("No Frames Detected")
            break
        if not detected:
            print(f"Frames detected : {totalFrames}")
            detected = True
        frame = cv2.LUT(frame, gamma_table)
        out.write(frame)
    cap.release()
    out.release()
    cv2.destroyAllWindows()

### Video Restoration by deblurring

In [ ]:
def RestoreFramesAndCreateVideo(videoName, fps, inputFrameDirectory, outputDirectory):
    print(f"Restoring the frames of {videoName}")

    !cd ./drive/MyDrive/BTP/RVRT && python3 main_test_rvrt.py --task 004_RVRT_videodeblurring_DVD_16frames --folder_lq /content/drive/MyDrive/BTP/Frames/VaibhavGeneratedMoviesFrames --tile 0 256 256 --tile_overlap 0 20 20 --num_workers 8 --save_video /content/drive/MyDrive/BTP/datasets/movies_dataset/VaibhavDataset/RestoredData --fps 25

### Frame Generation from clips

In [ ]:
def GenerateFrames(videoName, inputVideo, frameOutputDirectory):
    cap = cv2.VideoCapture(inputVideo)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    frameOutputPath = os.path.join(frameOutputDirectory, videoName)
    if not os.path.exists(frameOutputPath):
        os.makedirs(frameOutputPath)

    for j in range(frameCount):
        cap.set(cv2.CAP_PROP_POS_FRAMES, j)
        ret, frame = cap.read()
        if not ret:
            break

        framePath = f"{frameOutputPath}/frame_{j}.png"
        cv2.imwrite(framePath, frame)

    cap.release()
    cv2.destroyAllWindows()

### IQA + VR Layer

In [ ]:
def QualityCheck(videoName, frameOutputDirectory, videoRestoredDirectory, fps, iqa_metric, thresholdQuality=0.6, qualityCheckFrameSize=5):

    frameOutputPath = os.path.join(frameOutputDirectory, videoName)

    frameScores = []

    for j in range(qualityCheckFrameSize):
        framePath = f"{frameOutputPath}/frame_{j}.png"
        # Extract the score from the output
        score = iqa_metric(f'{framePath}').item()
        print(f"Score for Frame : {j + 1} is {score}")
        frameScores.append(score)

    avgScore = sum(frameScores) / len(frameScores)

    if avgScore < thresholdQuality:
        print(f"Video {videoName} with average score {avgScore} is below the threshold quality. Processing further.")
        # Image restoration begins
        RestoreFramesAndCreateVideo(videoName, fps, frameOutputDirectory, videoRestoredDirectory)

### Base Script

In [ ]:
videoGammaInputDirectory = f"./drive/MyDrive/BTP/datasets/{DATASET}_dataset/VaibhavDataset/OriginalData/"
videoGammaOutputDirectory = f"./drive/MyDrive/BTP/datasets/{DATASET}_dataset/VaibhavDataset/GammaData/"
frameOutputDirectory = f"./drive/MyDrive/BTP/Frames/VaibhavGeneratedMoviesFrames/"
videoRestoredDirectory = f"./drive/MyDrive/BTP/datasets/{DATASET}_dataset/VaibhavDataset/RestoredData/"
videoFinalSaveDirectory = f"./drive/MyDrive/BTP/datasets/{DATASET}_dataset/VaibhavDataset/OpenposeData/"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
fps = 25

# Gamma Corrected clip generation
for file in os.listdir(videoGammaInputDirectory):
    if file.endswith(".avi"):
        inputGammaVideoPath = os.path.join(videoGammaInputDirectory, file)
        videoName = os.path.splitext(file)[0]
        if GAMMA > 0:
            ProcessGammaVideo(videoName, inputGammaVideoPath, videoGammaOutputDirectory)

# Splitting of clips into frames for IQA
for file in os.listdir(videoGammaOutputDirectory):
    if file.endswith(".avi"):
        videoName = os.path.splitext(file)[0]
        print(f'Generating frames for {videoName}')
        videoName = os.path.splitext(file)[0]
        inputVideo = os.path.join(videoGammaOutputDirectory, file)
        GenerateFrames(videoName, inputVideo, frameOutputDirectory)
        print('Frames generated')

# IQA + VR
# The quality assessed for one video is used as a parameter for the entire dataset
iqa_metric = pyiqa.create_metric('topiq_nr', device = device)
for file in os.listdir(frameOutputDirectory):
    videoName = os.path.splitext(file)[0]
    print(f'Initializing quality check for {videoName}')
    QualityCheck(videoName, frameOutputDirectory, videoRestoredDirectory, fps, iqa_metric, 0.6, 5)
    break
# Add the above section to the main_test_rvrt.py file to test the scores for each video clip

### Openpose - Human Skeleton Generation

In [ ]:
# Openpose 
for file in os.listdir(videoRestoredDirectory):
    if file.endswith(".avi"):
        os.system(f'cd openpose && ./build/examples/openpose/openpose.bin --video ../{videoRestoredDirectory}{file} --display 0 --write_video ../{videoFinalSaveDirectory}{file} --disable_blending > /dev/null')